# 第一步：提取视频中的音频

The following script is adopted from
https://zhuanlan.zhihu.com/p/57944345

`glob.glob()`

返回文件的路径。若使用 `*` 描述文件名，则返回所有匹配文件的路径。

In [1]:
import glob
import os

video_dir = '/Users/wenjiazhai/Documents/GitHub/data_science_study/work/data/records'  

# 视频文件所在文件夹
os.chdir(video_dir)
# os.chdir(path): 改变 working directory 到 path

for directory in ['train', 'val', 'test']:
    if os.path.exists(directory):
        next
    else:
        os.mkdir(directory)
# 创建存储数据集的文件夹，备用

files = glob.glob(video_dir + '/*.mp4')
files[:5]

['/Users/wenjiazhai/Documents/GitHub/data_science_study/work/data/records/陀螺为什么不会倒.mp4',
 '/Users/wenjiazhai/Documents/GitHub/data_science_study/work/data/records/《流浪地球》科普答疑：人们为什么要去比邻星？太阳会爆炸吗？李永乐老师告诉你.mp4',
 '/Users/wenjiazhai/Documents/GitHub/data_science_study/work/data/records/什么是爱情？怎么谈恋爱，才能有效的找到自己的真命天子？李永乐老师讲爱情数学.mp4',
 '/Users/wenjiazhai/Documents/GitHub/data_science_study/work/data/records/光纤：光为什么能通讯？高锟为啥能得诺贝尔奖？李永乐老师追忆光纤之父.mp4',
 '/Users/wenjiazhai/Documents/GitHub/data_science_study/work/data/records/基因工程对人类的重大贡献：胰岛素是如何制作的？李永乐老师讲糖尿病医疗简史.mp4']

# 一、安装 pydub 库和 FFmpeg

MacOS 下可以使用 HomeBrew 安装 ffmpeg。

In [2]:
# 安装 Homebrew
# !/usr/bin/ruby -e "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/master/install)"

# 安装 ffmpeg
# !brew install ffmpeg

# 安装 pydub
# !pip install pydub

# 临时添加库的 path 到 Python 环境中
# https://stackoverflow.com/questions/57350259/filenotfounderror-errno-2-no-such-file-or-directory-ffprobe-ffprobe/57350596#57350596
import sys
sys.path.append('/path/to/ffmpeg')

# 二、抽取音频
考虑到所需抽取音频的数量较多，我们将遍历目录下所有视频文件，进行批量音频提取。

In [3]:
from pydub import AudioSegment

for video in files:
    filename = os.path.splitext(os.path.basename(video))[0]
    # os.path.splitext(path): Split the pathname path into a pair (root, ext) such that root + ext == path
    # os.path.basename(path): 返回指定文件的文件名
    wav_filename = filename + '.wav'
    if os.path.exists(wav_filename):
        # os.path.exists(path): 如果存在该文件或文件夹，返回 True
        next
    else:
        sound = AudioSegment.from_file(video).set_channels(1)
        sound = sound.set_frame_rate(16000)
        sound.export(wav_filename, format='wav')
        # https://blog.csdn.net/Debatrix/article/details/59058762
        # https://github.com/jiaaro/pydub/blob/master/API.markdown

In [4]:
audio_files = glob.glob('*.wav')
audio_files[:5]

['粒子加速器有什么用.wav',
 '输了就加倍下注，能赚钱吗？股票下跌就补仓？李永乐老师讲赌徒谬误.wav',
 '太阳会死亡吗？红巨星、中子星和黑洞等都是哪来的？李永乐老师讲恒星演化.wav',
 '李永乐老师讲宇称不守恒.wav',
 '凯利公式是啥？按这个炒股能成巴菲特？如何分配手里的钱进行最优投资，李永乐老师告诉你.wav']

# 第二步：划分数据集

现在手上已经有了 100 条音频，我们打算以 8:1:1 (training:validation:test) 的比例划分数据集。

假定音频的大小与时长成正比，我们希望找出最短的 80 条音频作为训练集，10 条中等长度的音频作为验证集，10 条最长的音频作为测试集。

`os.path.getsize()`

返回文件的大小。

In [5]:
file_size = []
for file in audio_files:
    file_size.append(os.path.getsize(file))
print(file_size[:10])

[35985402, 19215022, 24162914, 32166926, 22176772, 26944852, 25515988, 29692608, 19752238, 29134586]


In [6]:
# 使用 `zip` 将文件名与相应的大小放在一起，以文件大小排序，使用解析式提取文件名。
sorted_file_lists = [f for _, f in sorted(zip(file_size, audio_files))]
train_file_lists = sorted_file_lists[:80]
val_file_lists = sorted_file_lists[80:90]
test_file_lists = sorted_file_lists[90:]

print('训练集:', train_file_lists[:5])
print('验证集:', val_file_lists[:5])
print('测试集:', test_file_lists[:5])

训练集: ['地球的半径和质量都是怎么测量出来的.wav', '《流浪地球》科普答疑：人们为什么要去比邻星？太阳会爆炸吗？李永乐老师告诉你.wav', '量子延迟选择实验是怎么回事.wav', '断臂的维纳斯到底有多高.wav', '陀螺为什么不会倒.wav']
验证集: ['反物质是什么.wav', '李永乐老师讲音律.wav', '海森堡不确定性原理和量子隧穿效应如何理解.wav', '外星人存在吗？人类为什么看不到外星文明？李永乐老师讲费米悖论.wav', 'ABO血型有啥区别？孩子是不是亲生，能用血型判断吗？李永乐老师讲造血干细胞移植.wav']
测试集: ['0.999…=1？数到底是什么？李永乐老师讲数学公理化.wav', '粒子加速器有什么用.wav', '李永乐老师教你安全使用电池.wav', '首张黑洞照片咋拍的？事件视界望远镜EHT是什么.wav', '李永乐老师讲碳的同素异形体.wav']


# 第三步

## 构建训练集、测试集:
1. 根据字幕切分音频
2. 字幕转换成拼音，放在字幕前面
3. 将新文件名放在拼音前面，将所有文本打包成一个文件，与新文件放在一起。

In [7]:
# 前期的准备工作，安装拼音功能库
# !pip install pinyin

In [8]:
_MAPPING = (u'零', u'一', u'二', u'三', u'四', u'五', u'六', u'七', u'八', u'九', u'十', 
            u'十一', u'十二', u'十三', u'十四', u'十五', u'十六', u'十七',u'十八', u'十九')
_P0 = (u'', u'十', u'百', u'千', u'万')
_S4 = 100000

# 不涉及数位的数字转汉字函数

def frac2chn(line):
    match = re.findall('\.\d+', line)
    if not match:
        pass
    else:
        new = [_MAPPING[w] for w in match[0]]
        new = ''.join(new)
        line = re.sub(match[0], new, line)
        line = re.sub('.', '点', line)
    return line

# 涉及数位的数字转汉字函数
# https://www.jianshu.com/p/c87581f9aaa4

def _to_chinese(num):
    assert (0 <= num and num < _S4), f'{num} is out of range'
    if num < 20:
        return _MAPPING[num]
    else:
        lst = []
        while num >= 10:
            lst.append(num % 10)
            num = num // 10
        lst.append(num)
        c = len(lst)  # 位数
        result = u''

        for idx, val in enumerate(lst):
            val = int(val)
            if val != 0:
                result += _P0[idx] + _MAPPING[val]
                if idx < c - 1 and lst[idx + 1] == 0:
                    result += u'零'
        result = result[::-1]
    return result

def num2chn(line):
    match = re.findall('\d+', line)
    sub = [_to_chinese(int(s)) for s in match]
    for i in range(len(match)):
        line = re.sub(match[i], sub[i], line)
    return line

In [9]:
import re
from os import listdir
from os.path import isfile, join
import pinyin
from tqdm import tqdm

def built_dataset(data, path):
    length = len(data)
    new_num = 0
    for i, file in enumerate(data):
        print(f'处理第 {i+1} / {length} 文件：',file)
        piece_audio = AudioSegment.from_wav(file)
        
        sub_file = file[:-4] + '.srt'
        
        # 读取字幕文件，分离时间戳和字幕
        with open(sub_file, 'r') as f:
            raw = f.readlines()
        for line in raw:
            line = line.replace('\n', '')
        raw = [re.sub('[\n\s]', '', l) for l in raw]
        timestamp_init = raw[1::4]
        subtitle_init = raw[2::4][:-2]
        
        # 过滤字幕，去掉带特殊字符的字幕（如 ‘√’）和相应的时间戳
        timestamp = []
        subtitle_raw = []
        for i in range(len(subtitle_init)):
            if not re.findall('\W', subtitle_init[i]):
                subtitle_raw.append(subtitle_init[i])
                timestamp.append(timestamp_init[i])
        
        # 处理字幕，将阿拉伯数字转换成汉字(105.10 -> 一百零五点一零），并用空格隔开每个字
        # 数字转汉字分两步：首先转换小数点后面的部分，再转换小数点前面的部分
        subtitle_raw = [frac2chn(l) for l in subtitle_raw]
        subtitle_raw = [num2chn(l) for l in subtitle_raw]
        subtitle = [' '.join(l) for l in subtitle_raw]
        
        # 处理时间戳
        recording = []
        for time in timestamp:
            t = re.findall('00:(\d+):(\d+),(\d+)-->00:(\d+):(\d+),(\d+)', time)[0]
            start = int(t[0]) * 60 * 1000 + int(t[1]) * 1000 + int(t[2])
            end = int(t[3]) * 60000 + int(t[4]) * 1000 + int(t[5])
            recording.append((start, end))
        
        # 提取音频，生成一个训练文件，同时将音频文件名、字幕、拼音写入同一个文件作为一个标注
        output_filepath = video_dir + '/'+ path + '/' # 输出文件路径
        
        # 提取文件名中最大的编号
        file_list = [f for f in listdir(output_filepath) if isfile(join(output_filepath, f))]
        nums = [re.findall(r'^(\d+)', f) for f in file_list if f.endswith('.wav')]
        # https://stackoverflow.com/questions/3207219/how-do-i-list-all-files-of-a-directory
        if not nums:
            new_num = 0
        else:
            new_num = max([int(f[0]) for f in nums]) + 1
        print(f'抓取第 {new_num} 段音频')
        
        for j, sub in enumerate(subtitle):
            output_filename = path + '/' + str(new_num) + ".wav"
            piece = piece_audio[recording[j][0]:recording[j][1]]
            piece.export(output_filename, 'wav')
            label_filename = path + '/' + str(new_num) + '.txt'
            
            with open(label_filename, 'w') as f:
                f.write(output_filename + '\n')
                f.write(pinyin.get(sub, format='numerical') + '\n')
                f.write(sub)
            
            new_num += 1

In [10]:
# 创建训练集
built_dataset(train_file_lists, 'train')

处理第 1 / 80 文件： 地球的半径和质量都是怎么测量出来的.wav
抓取第 0 段音频
处理第 2 / 80 文件： 《流浪地球》科普答疑：人们为什么要去比邻星？太阳会爆炸吗？李永乐老师告诉你.wav
抓取第 97 段音频
处理第 3 / 80 文件： 量子延迟选择实验是怎么回事.wav
抓取第 198 段音频
处理第 4 / 80 文件： 断臂的维纳斯到底有多高.wav
抓取第 315 段音频
处理第 5 / 80 文件： 陀螺为什么不会倒.wav
抓取第 422 段音频
处理第 6 / 80 文件： 李永乐老师讲条件概率的贝叶斯公式.wav
抓取第 592 段音频
处理第 7 / 80 文件： 牛顿用木桶实验论证绝对时空，“以太”却被迈克耳孙和莫雷宣布死刑.wav
抓取第 713 段音频
处理第 8 / 80 文件： 为什么久赌必输？股票加杠杆，风险为啥这么大？李永乐老师讲赌徒输光原理.wav
抓取第 882 段音频
处理第 9 / 80 文件： 霍尔效应是什么？车速表是如何测量汽车速度的？李永乐老师讲霍尔传感器.wav
抓取第 992 段音频
处理第 10 / 80 文件： 帆船竟然可以逆风航行？而且还有圆筒形的风帆？李永乐老师讲帆船的原理.wav
抓取第 1165 段音频
处理第 11 / 80 文件： 矿泉水从瓶里流出需要多长时间？这个问题并不简单！.wav
抓取第 1335 段音频
处理第 12 / 80 文件： 李永乐老师讲力矩平衡（1）.wav
抓取第 1527 段音频
处理第 13 / 80 文件： 李永乐老师讲力矩平衡（2）.wav
抓取第 1744 段音频
处理第 14 / 80 文件： 仰望同一片星空1:3.wav
抓取第 1961 段音频
处理第 15 / 80 文件： 人推墙壁没推动，不做功但为啥也会累？肌肉是咋工作的？李永乐老师告诉你.wav
抓取第 2205 段音频
处理第 16 / 80 文件： 输了就加倍下注，能赚钱吗？股票下跌就补仓？李永乐老师讲赌徒谬误.wav
抓取第 2366 段音频
处理第 17 / 80 文件： 如何说话让人喜欢？怎么做生意更吸引顾客？李永乐老师讲框架效应.wav
抓取第 2565 段音频
处理第 18 / 80 文件： 传销不挣钱为啥还有这么多人参与？美国大选为啥总是两党之争？李永乐老师讲一美元拍

In [11]:
built_dataset(val_file_lists, 'val')

处理第 1 / 10 文件： 反物质是什么.wav
抓取第 0 段音频
处理第 2 / 10 文件： 李永乐老师讲音律.wav
抓取第 451 段音频
处理第 3 / 10 文件： 海森堡不确定性原理和量子隧穿效应如何理解.wav
抓取第 780 段音频
处理第 4 / 10 文件： 外星人存在吗？人类为什么看不到外星文明？李永乐老师讲费米悖论.wav
抓取第 1161 段音频
处理第 5 / 10 文件： ABO血型有啥区别？孩子是不是亲生，能用血型判断吗？李永乐老师讲造血干细胞移植.wav
抓取第 1473 段音频
处理第 6 / 10 文件： 诺贝尔物理学奖深度解读：光学镊子和啁啾放大技术是什么？李永乐老师告诉你.wav
抓取第 1878 段音频
处理第 7 / 10 文件： 李永乐老师讲落猫问题.wav
抓取第 2221 段音频
处理第 8 / 10 文件： 耶鲁大学的实验推翻了量子力学吗.wav
抓取第 2628 段音频
处理第 9 / 10 文件： 李永乐老师讲机会成本.wav
抓取第 3060 段音频
处理第 10 / 10 文件： 经济泡沫1:4.wav
抓取第 3497 段音频
